In [1]:
from tqdm import tqdm
import os
from glob import glob

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_dir = "annotated_data"

def to_keep(ID):
    first = ID[0] in {"A", "T"}
    try:
        int(ID[1])
        return first
    except ValueError:
        return False

def parse_middle(row):
    if row.name[1].startswith("T"):
        ent_type, span = row.middle.split(" ", 1)
        return {"entity_type": ent_type, "span": span}
    else:
        keys = ["attr_type", "attr_ref", "attr_val"]
        return dict(zip(keys, str(row.middle).split()))

def df_from_ann_file(f):
    df = pd.read_csv(f, sep="\t", header=None, 
                         names=["ID", "middle", "entity_name"])
    df = df[df.ID.apply(to_keep)]
    df["file"] = f
    df = df.set_index(["file", "ID"])
    mdf = pd.DataFrame.from_records(df.apply(parse_middle, axis="columns")).set_index(df.index)

    return df.join(mdf).drop("middle", axis=1)
    return pd.concat([df, mdf], axis=1)


In [3]:
done = set()

dfs = []
pbar = tqdm(set(glob(f"{data_dir}/*/*.ann")))
for f in pbar:
    pbar.set_description(f)
    file = f.split("/")[-1]
    if file in done:
        continue
    done.add(file)    
    df = df_from_ann_file(f)
    
    dfs.append(df)

annotated_data/E/NL-HaNA_1.04.02_6869_0081.ann: 100%|█| 2199/2199 [00:18<00:00, 


In [4]:
big = pd.concat(dfs)
big = filter_df(big)

NameError: name 'filter_df' is not defined

In [ ]:
# Organisation needs attr Beneficiary
    
def is_bad_org(t, sub_df):
    atts = sub_df[sub_df.attr_ref == t]
    return atts[atts.attr_type == "Beneficiary"].shape[0] == 0

ls = []
for file, sub in tqdm(big.groupby(level=0)):    
    orgs = sub[sub.entity_type == "Organization"]
    for (file2, row_id), r in orgs.iterrows():
        bad  = is_bad_org(row_id, sub)
        if bad:
            sub = sub.drop((file2, row_id))
        else:
            print(file, sub.shape)
        
    ls.append(sub)
new_big = pd.concat(ls)

In [ ]:
# Person needs Role AND LegalStatus AND Gender
    
def is_bad_pers(t, sub_df):
    atts = sub_df[sub_df.attr_ref == t]
    return (atts[atts.attr_type == "Gender"].shape[0] == 0 or
            atts[atts.attr_type == "Role"].shape[0] == 0 or
            atts[atts.attr_type == "LegalStatus"].shape[0] == 0)

ls = []
for file, sub in tqdm(new_big.groupby(level=0)):    
    pers = sub[sub.entity_type == "Person"]
    for (file2, row_id), r in pers.iterrows():
        bad  = is_bad_pers(row_id, sub)
        if bad:
            sub = sub.drop((file2, row_id))
            
            have_this_id = sub[sub.attr_ref == row_id]
            sub = sub.drop(have_this_id.index)
        else:
            pass
#             print(file, sub.shape)
        
    ls.append(sub)
new_big = pd.concat(ls)

In [ ]:
big = new_big

In [ ]:
def barplot(x, y, rotation=0):
    plt.figure(figsize=(8, 8))
    plt.bar(x, y)
    # plt.plot(x, y, ".", )
    for cur_x, cur_y in zip(x, y): plt.annotate(round(cur_y, 3), (cur_x, cur_y+0.01))
    _ = plt.xticks(x, y.index, rotation=rotation)
    plt.ylim((0, 1))
    plt.show()

In [ ]:
print(f"1. # unique pages annotated:\t{len(done)}")

In [ ]:
print(f"3.1 # of annotations (= # rows in table):\t{big.shape[0]}")
n = big[big.index.get_level_values(1).str.startswith("T")].shape[0]
print(f"3.1 # of entity annotations (= # rows in table starting with 'T'):\t{n}")
print(f"3.1 # of attribute annotations\t{big.shape[0] - n}")

In [ ]:
# print(f"# spans:\t{(~big.span.isna()).sum()}")

print(f"3.2 # tokens(!) in entity names:\t{big.entity_name.fillna('').str.split().apply(len).sum()}")

In [ ]:
ns = big.groupby("entity_type").apply(lambda sub: (~sub.span.isna()).sum())

print("\n3.3 number of entities per type".upper())
print(ns)
print()
# print(big.groupby("entity_type").apply(lambda sub: (~sub.span.isna()).sum())/sub.shape[0])
print("3.3 percentage of entities of type".upper())
print((ns/ns.sum()))

# x, y = [1,2,3, 4], (ns/ns.sum())
# plt.figure(figsize=(8, 8))
# plt.bar(x, y)
# # plt.plot(x, y, ".", )
# for cur_x, cur_y in zip(x, y): plt.annotate(round(cur_y, 3), (cur_x, cur_y+0.01))
# _ = plt.xticks(x, y.index)
# plt.ylim((0, 1))
# plt.show()

barplot([1,2,3, 4], (ns/ns.sum()))



ns_toks = big.groupby("entity_type").apply(lambda sub:
                                     sub.entity_name.fillna('').str.split().apply(len).sum())

print("\n3.4 number of tokens per entity type".upper())
print(ns_toks)
print("\n3.4 average number of tokens of each entity per entity type".upper())
print(ns_toks/ns)

In [ ]:
for at, sub in big.groupby("attr_type"):
    print(f"4. {at.upper()}")
    cs = sub.attr_val.value_counts()
    print(cs)
    print()
    print(cs/cs.sum())
    
    barplot(list(range(1, cs.shape[0]+1)),
           cs/cs.sum(), rotation=(360-35 if cs.shape[0] > 6 else 0))
    
    print()
#     sub.attr_val.hist(bins=20)
#     plt.title(at)
#     plt.show()


# big[big.attr_type == "Role"].attr_val.value_counts()